In [ ]:
# !pip install -r requirements.txt

In [ ]:
import subprocess
import shutil

In [ ]:
# Clone the repository
subprocess.run(["git", "clone", "https://huggingface.co/datasets/dreamerdeo/multispider"], check=True)

# Move the directories
shutil.move("multispider/dataset", ".")
shutil.move("multispider/model", ".")

# Prepare the dataset

We're gonna separate the dataset into the folder of the different databases, to reduce the size of the dataset when working a given database.

We will keep the distinction between dev, train and examples sets

In [16]:
dataset_path = r"../multispider/multispider/dataset/multispider/with_original_value"
database_path = r"../multispider/multispider/dataset/spider/database"

On crée une liste des databases

Pour chaque fichier dans la liste suivante [dev_fr.json, train_fr.json] on effectue le traitement suivant:
- On ouvre le fichier
- on crée un dictionnaire avec comme clefs les noms des bases de données et une chaine de caractère vide comme valeur
- On parcourt les lignes du fichier Json en cherchant la partie "db_id"
- On ajoute la ligne dans le dictionnaire correspondant à la base de données
- Puis une fois le fichier parcouru, on écrit les lignes dans un nouveau fichier json dans le dossier correspondant à la base de données pour chaque base de données dans le dictionnaire, en conservant le nom du fichier original



In [44]:
import json
import os

def extract_database(file_path):
    with open(os.path.join(dataset_path, file_path), "r", encoding="utf-8") as file:
        data = json.load(file)
        databases = {}
        for line in data:
            db_id = line["db_id"]
            if db_id not in databases:
                databases[db_id] = []
            databases[db_id].append(line)
    return databases

def write_database(databases, file_path):
    for db_id, lines in databases.items():
        with open(os.path.join(database_path, db_id, file_path), "w", encoding="utf-8") as file:
            json.dump(lines, file)
            
def extract_and_write_database(file_path):
    databases = extract_database(file_path)
    write_database(databases, file_path)
    
def extract_all_databases():
    for file in ["dev_fr.json", "train_fr.json"]:
        extract_and_write_database(file)
        

In [45]:
extract_all_databases()

In [24]:
# On fait la liste de toutes les bases de données avec un fichier train_fr.json, et celles avec un fichier dev_fr.json

def list_databases(file_path):
    with open(os.path.join(dataset_path, file_path), "r") as file:
        data = json.load(file)
        databases = set()
        for line in data:
            db_id = line["db_id"]
            databases.add(db_id)
    return databases

def list_all_databases():
    databases_dev = list_databases("dev_fr.json")
    databases_train = list_databases("train_fr.json")
    return databases_dev, databases_train

databases_dev, databases_train = list_all_databases()

In [25]:
# on cherche les databases qui sont dans les deux listes
databases_dev & databases_train

set()

In [26]:
# On enregistre les bases de données dans des fichiers JSON
with open("databases_dev.json", "w") as file:
    json.dump(list(databases_dev), file)

with open("databases_train.json", "w") as file:
    json.dump(list(databases_train), file)